#Thanks to: https://pythonology.eu/a-rag-web-scraper-with-langchain-ollama-and-chroma/

Getting repo readme context, applied to:

1. https://github.com/JAlcocerT/Streamlit-MultiChat
2. https://github.com/qatrackplus/qatrackplus/

In [3]:
# !pip install requests
# !pip install beautifulsoup4
# !pip install langchain-community


In [2]:
!pip show langchain

Name: langchain
Version: 0.3.25
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: async-timeout, langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community


In [ ]:
#brings your OPENAI_API_KEY
!source .env

In this step, we’re using **BeautifulSoup’s SoupStrainer** to focus on the “content-area” class, ensuring we only extract relevant information. 

The WebBaseLoader then fetches and processes the specified URL.

In [6]:
# class="markdown-body entry-content container-lg"

import bs4
from langchain_community.document_loaders import WebBaseLoader

bs4_strainer = bs4.SoupStrainer(class_=("markdown-body entry-content container-lg"))
loader = WebBaseLoader(
    web_paths=("https://github.com/JAlcocerT/Streamlit-MultiChat",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [7]:
!pip show langchain-text-splitters

Name: langchain-text-splitters
Version: 0.3.8
Summary: LangChain text splitting utilities
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: langchain-core
Required-by: langchain


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

Get Ollama ready:

* https://github.com/JAlcocerT/Docker/tree/main/AI_Gen/Ollama


```sh
sudo docker stats ollama
```

In [ ]:
# !docker exec -it ollama sh
#hostname -I 
#--->> 172.20.0.2 
#sudo docker inspect ollama

In [12]:
# !ollama run llama3.2:1b

In [13]:
# !ollama pull all-minilm

In [ ]:
# !pip install langchain-chroma
# #https://pypi.org/project/langchain-chroma/

In [17]:
!pip show langchain-chroma

Name: langchain-chroma
Version: 0.2.3
Summary: An integration package connecting Chroma and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: chromadb, langchain-core, numpy
Required-by: 


In [19]:
###DEPRICATED VERSION OF DOING IT###

# from langchain_community.embeddings import OllamaEmbeddings #https://python.langchain.com/docs/integrations/text_embedding/ollama/
# from langchain_chroma import Chroma

# # local_embeddings = OllamaEmbeddings(model="all-minilm")
# # vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

# from langchain_community.embeddings import OllamaEmbeddings
# #local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://192.168.0.12:11434")
# #local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://192.168.1.5:11434")
# local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://0.0.0.0:11434")

/tmp/ipykernel_535655/1148073819.py:10: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://0.0.0.0:11434")


In [21]:
# #https://pypi.org/project/langchain-ollama/
# !pip install langchain-ollama==0.3.2

In [22]:
!pip show langchain-ollama

Name: langchain-ollama
Version: 0.3.2
Summary: An integration package connecting Ollama and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: langchain-core, ollama
Required-by: 


In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

#Using a local embedding model
#https://ollama.com/library/all-minilm

# local_embeddings = OllamaEmbeddings(model="all-minilm")
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

# local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://192.168.0.12:11434")
# local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://192.168.1.5:11434")
local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://0.0.0.0:11434")

In [24]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)


In [67]:
question = "what is the repository about?"
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke(question)

In [66]:
context = ' '.join([doc.page_content for doc in retrieved_docs])


In [27]:
context

'Streamlit-MultiChat\n\n\nMany LLMs - One Streamlit Web App\n\n\nOpenAI | Anthropic | Ollama | Groq\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nA custom Streamlit Web App to Chat with the latest LLMs and get a per use cost instead of a fixed monthly price.\nFeatures\nUse many large language models: OpenAI, Anthropic, Open / Local LLM\'s with one Streamlit Web App.\n\nLLM Support\n\nOllama - Open Source Models\nOpenAI - GPT 3.5 / GPT4 / GPT4o / GPT4o-mini\nAnthropic - Claude 3 (Opus / Sonnet) / Claude 3.5\nGroq API - LlaMa models using quick LPU inference\n\n\nExtended explanation\n\nSliDev presentation of the Streamlit-MultiChat\nThis blog post →\nDeploy as per https://github.com/JAlcocerT/Streamlit-MultiChat/tree/main/Z_DeployMe\n\n\n\nDuring the process, I also explored: SliDev PPTs, ScrapeGraph, DaLLe, Streamlit Auth and OpenAI as Custom Agents\nGetting Started\nThe Project is documented here →\n\nClone the repository and Run with your API keys 👇\n  \xa0\n\nOpenAI A

In [29]:
# !pip install -U langchain-ollama
#0.3.1

In [ ]:
#https://python.langchain.com/api_reference/ollama/index.html

# from langchain_ollama.llms import OllamaLLM
# llm = OllamaLLM(model="llama3.2:1b")

from langchain_ollama.llms import OllamaLLM

#!ollama pull llama3.2:1b
#!ollama list
llm = OllamaLLM(model="llama3.2:1b", base_url="http://0.0.0.0:11434")
#llm = OllamaLLM(model="llama3.2:1b", base_url="http://192.168.0.12:11434") #workin with LLama3.2
#llm = OllamaLLM(model="all-minilm", base_url="http://0.0.0.0:11434") #not working with this model, as it is a embedding model!!!


response = llm.invoke(f"""Answer the question according to the context given very briefly:
           Question: {question}.
           Context: {context}
""")

In [20]:
response

'There are several oversold and overbought periods:\n\n- Overbought: The Relative Strength Index (RSI) is above 70 (30-70)\n- Oversold: The Relative Strength Index (RSI) is below 30 (70-30)\n\nThese levels can be used to generate buy or sell signals based on technical analysis strategies.'

In [37]:
# !pip install langchain_openai==0.3.16

In [ ]:
#export OPENAI_API_KEY="your-api-key"
#source .env

In [42]:
import openai
import os

# Configure your OpenAI API key (if not set as environment variable)
# openai.api_key = "YOUR_OPENAI_API_KEY"

try:
    models = openai.models.list()
    model_names = [model.id for model in models.data]
    print("Available OpenAI Models:")
    for name in model_names:
        print(f"- {name}")
except openai.error.AuthenticationError as e:
    print(f"Authentication error: {e}")
    print("Please ensure your OPENAI_API_KEY environment variable is set correctly.")
except openai.error.OpenAIError as e:
    print(f"An OpenAI API error occurred: {e}")

Available OpenAI Models:
- gpt-4o-audio-preview-2024-12-17
- dall-e-3
- dall-e-2
- gpt-4o-audio-preview-2024-10-01
- gpt-4-turbo-preview
- text-embedding-3-small
- gpt-4-turbo
- gpt-4-turbo-2024-04-09
- gpt-4.1-nano
- gpt-4.1-nano-2025-04-14
- gpt-4o-realtime-preview-2024-10-01
- gpt-4o-realtime-preview
- babbage-002
- gpt-4
- text-embedding-ada-002
- chatgpt-4o-latest
- gpt-4o-realtime-preview-2024-12-17
- gpt-4o-mini-audio-preview
- gpt-4o-audio-preview
- o1-preview-2024-09-12
- gpt-4o-mini-realtime-preview
- gpt-4.1-mini
- gpt-4o-mini-realtime-preview-2024-12-17
- gpt-3.5-turbo-instruct-0914
- gpt-4o-mini-search-preview
- gpt-4.1-mini-2025-04-14
- o1
- o1-2024-12-17
- davinci-002
- gpt-3.5-turbo-1106
- gpt-4o-search-preview
- gpt-4-1106-preview
- gpt-3.5-turbo-instruct
- gpt-3.5-turbo
- gpt-4o-mini-search-preview-2025-03-11
- gpt-4-0125-preview
- gpt-4o-2024-11-20
- whisper-1
- gpt-4o-2024-05-13
- o1-pro
- gpt-3.5-turbo-16k
- gpt-image-1
- o1-pro-2025-03-19
- o1-preview
- gpt-4-0613

In [43]:
from langchain_openai.llms import OpenAI

# You'll need your OpenAI API key
# It can be set as an environment variable OPENAI_API_KEY=your_key
# Or passed directly as a parameter:



#llm = OpenAI(model_name="gpt-3.5-turbo-instruct") # Or any other OpenAI model
llm = OpenAI(model_name="gpt-4.1-nano")

response = llm.invoke(f"""Answer the question according to the context given very briefly:
           Question: {question}.
           Context: {context}""")

In [35]:
!pip show langchain_openai

Name: langchain-openai
Version: 0.3.16
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


In [44]:
response

'. The repository is about a Streamlit web app that allows users to chat with multiple large language models (LLMs) from various providers like OpenAI, Anthropic, Ollama, and Groq, and compare costs on a per-use basis.'

In [ ]:
# !pip install groq==0.24.0

In [49]:
!pip show groq

Name: groq
Version: 0.24.0
Summary: The official Python library for the groq API
Home-page: https://github.com/groq/groq-python
Author: 
Author-email: Groq <support@groq.com>
License: Apache-2.0
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, typing-extensions
Required-by: 


In [ ]:
#!source .env
#export GROQ_API_KEY="your-api-key"

In [ ]:
from groq import Groq
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

try:
    # Get the Groq API key from the environment variables
    groq_api_key = os.getenv("GROQ_API_KEY")

    if not groq_api_key:
        raise ValueError("GROQ_API_KEY not found in environment variables or .env file.")

    # Initialize the Groq client with the API key
    groq_client = Groq(api_key=groq_api_key)

    models = groq_client.models.list()
    model_names = [model.id for model in models.data]
    print("Available Groq Models:")
    for name in model_names:
        print(f"- {name}")

except ValueError as ve:
    print(f"Configuration Error: {ve}")
    print("Please ensure you have a .env file with GROQ_API_KEY set.")
except Exception as e:
    print(f"An error occurred while fetching Groq models: {e}")
    print("Please ensure you have the 'groq' library installed and your GROQ_API_KEY is correctly configured.")

Available Groq Models:
- llama-3.1-8b-instant
- allam-2-7b
- llama-3.3-70b-versatile
- meta-llama/llama-4-scout-17b-16e-instruct
- compound-beta
- llama3-70b-8192
- whisper-large-v3
- distil-whisper-large-v3-en
- playai-tts
- playai-tts-arabic
- llama3-8b-8192
- qwen-qwq-32b
- deepseek-r1-distill-llama-70b
- mistral-saba-24b
- llama-guard-3-8b
- compound-beta-mini
- gemma2-9b-it
- whisper-large-v3-turbo
- meta-llama/llama-4-maverick-17b-128e-instruct


In [61]:
# !pip install langchain-groq==0.3.2

In [60]:
!pip show langchain-groq

Name: langchain-groq
Version: 0.3.2
Summary: An integration package connecting Groq and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: groq, langchain-core
Required-by: 


In [68]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the Groq API key from the environment variables
groq_api_key = os.getenv("GROQ_API_KEY")

if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in environment variables or .env file.")

# Initialize the Groq chat model
llm = ChatGroq(
    model_name="llama-3.1-8b-instant",  # Or another available Groq model
    groq_api_key=groq_api_key
)

# Example usage (assuming 'question' and 'context' variables are defined)
#question = "What is the capital of Poland?"
#context = "Warsaw is the largest city and capital of Poland."

question = "what is the repository about?"
context = ' '.join([doc.page_content for doc in retrieved_docs])


try:
    response = llm.invoke(f"""Answer the question according to the context given very briefly:
               Question: {question}.
               Context: {context}""")
    print(response)
except Exception as e:
    print(f"An error occurred during inference: {e}")

content='The repository is about a custom Streamlit Web App that allows users to chat with various large language models (LLMs) from different providers, including OpenAI, Anthropic, Ollama, and Groq, at a per-use cost instead of a fixed monthly price.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 854, 'total_tokens': 910, 'completion_time': 0.074666667, 'prompt_time': 0.101103705, 'queue_time': 1.122559132, 'total_time': 0.175770372}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f7bd09b454', 'finish_reason': 'stop', 'logprobs': None} id='run--2d780429-828b-4ae7-b5ad-3f05e12fd9cc-0' usage_metadata={'input_tokens': 854, 'output_tokens': 56, 'total_tokens': 910}


In [69]:
type(response)

langchain_core.messages.ai.AIMessage

In [70]:
print(response.content)  # Access the generated text content

The repository is about a custom Streamlit Web App that allows users to chat with various large language models (LLMs) from different providers, including OpenAI, Anthropic, Ollama, and Groq, at a per-use cost instead of a fixed monthly price.


In [45]:
# !pip freeze > requirements-export.txt